In [141]:
%load_ext autoreload
%load_ext iminizinc
%matplotlib inline
%autoreload 2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc


In [142]:
def read_input(path):
    file = open(path,"r").readlines()
    w_paper , h_paper = tuple(map(int,file[0].rstrip("\n").split(" ")))
    n = int(file[1].rstrip("\n"))
    presents = []
    for i in range(2,n+2):
        presents.append(list(map(int,file[i].rstrip("\n").split(" "))))
    return w_paper,h_paper,n,presents

In [143]:
path = "instances/35x35.txt"
w_paper, h_paper, n, presents = read_input(path)
bl_corners = []
presents

[[3, 3],
 [3, 4],
 [3, 5],
 [3, 6],
 [3, 7],
 [3, 8],
 [3, 9],
 [3, 10],
 [3, 12],
 [3, 19],
 [3, 22],
 [4, 3],
 [4, 4],
 [4, 5],
 [4, 6],
 [4, 7],
 [4, 8],
 [4, 12],
 [4, 17],
 [4, 20],
 [4, 23],
 [14, 35]]

In [144]:
%%mzn_model test
include "globals.mzn";
int: w_paper;
int: h_paper;
int: n;
array[1..n,1..2] of var 0..max(w_paper,h_paper)-1: bl_corners;
array[1..n,1..2] of int: presents;
constraint forall(i in 1..n)(bl_corners[i,1]+presents[i,1]<=w_paper);
constraint forall(i in 1..n)(bl_corners[i,2]+presents[i,2]<=h_paper);
constraint forall(i in 1..n,j in i+1..n)(
    max(bl_corners[i,1],bl_corners[j,1])>=min(bl_corners[i,1]+presents[i,1],bl_corners[j,1]+presents[j,1]) \/
    max(bl_corners[i,2],bl_corners[j,2])>=min(bl_corners[i,2]+presents[i,2],bl_corners[j,2]+presents[j,2])
    );
solve satisfy;

In [145]:
def show_solutions(w_paper,h_paper,n,presents,solutions):
    colours = np.random.rand(n,3)
    for x in solutions:
        bl_corners = x['bl_corners']
        show_solution(w_paper,h_paper,n,presents,bl_corners,colours)

def show_solution(w_paper,h_paper,n,presents,bl_corners,colours=None):
    fig, ax = plt.subplots()
    if colours is None:
        colours = np.random.rand(n,3)
    for i in range(n):
        ax.add_patch(patches.Rectangle(bl_corners[i],presents[i][0],presents[i][1],label=i,facecolor=colours[i],lw=4))
    ax.set_xlim(0, w_paper)
    ax.set_ylim(0, h_paper)
    plt.show()

solutions = %minizinc -a test
print(f"Number of solutions= {len(solutions)} ")
show_solutions(w_paper,h_paper,n,presents,solutions)

In [116]:
constraint forall(i in 1..n,j in i+1..n)(not( bl_corners[j,1]==bl_corners[i,1] /\ bl_corners[j,2]==bl_corners[i,2]));

constraint forall(i in 1..n,j in i+1..n)(if bl_corners[j,1]>bl_corners[i,1] then bl_corners[j,1]>=bl_corners[i,1]+presents[i,1] else true endif /\ if bl_corners[j,2]>bl_corners[i,2] then bl_corners[j,2]>=bl_corners[i,2]+presents[i,2] else true endif);

SyntaxError: invalid syntax (<ipython-input-116-095f6cebe787>, line 1)